In [88]:
import pandas as pd
from sklearn import datasets, linear_model
import numpy as np
from matplotlib import pyplot as plt

from operator import itemgetter
from math import sqrt

In [89]:
seadata = pd.read_csv('sea-level-data.csv', encoding = 'ISO-8859-1').ix[1:, :]

In [90]:
seadata

,Year,1992,1992.1,1992.2,1992.3,1992.4,1992.5,1992.6,1992.7,1992.8,...,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,2016.8,2016.9,2016.10,2016.11
1,1,-247,-351,-300,-142,-45,-226,-340,-316,-97,...,-212,-130,-90,-155,-237,-161,-23,111,136,0
2,2,-438,-99999,-399,-195,-143,-255,-429,-441,-225,...,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,0
3,3,-455,-556,-458,-290,-165,-293,-458,-464,-254,...,-317,-184,-24,-122,-264,-210,-82,30,XXXXX,0
4,4,-447,-548,-487,-289,-197,-362,-523,-508,-267,...,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,0
5,5,-483,-587,-495,-318,-209,-364,-507,-483,-251,...,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,0
6,6,-250,-360,-284,-113,-15,-186,-339,-326,-107,...,-265,-121,-1,-86,-183,-114,21,95,109,0
7,7,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,...,-239,-86,14,-48,-182,-145,-22,37,100,0
8,8,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,...,-231,-98,14,-92,-205,-142,7,100,123,0
9,9,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,XXXXX,...,-217,-91,12,-81,-211,-126,26,144,145,0
10,10,-330,-381,-302,-183,-74,-199,-281,-275,-135,...,-234,-124,-58,-97,-137,-70,-13,52,27,0


In [91]:
seadata.iloc[0, 1]

'-247'

# this part is used to find similarities

In [92]:
#the approach to fill the missing data is:
#1 using Pearson product-moment correlation coefficient to find strongly linear relationship blcok
#find the linear regression equation between this two block
#use this equation to fill out missing data

In [93]:
def find_mean(row):
    row_sum = 0
    row_valid = 0
    for data in row:
        if (data == "-99999" or data == "XXXXX"):
            pass
        else:
            row_sum += int(data)
            row_valid += 1
    return row_sum / row_valid

In [94]:
def find_sd(row, mean):
    var_sum = 0
    for data in row:
        if (data == "-99999" or data == "XXXXX"):
            pass
        else:
            var_sum += (int(data) - mean)**2
    
    return sqrt(var_sum)

In [95]:
def similarities(block1, block2):
    #Pearson correlation coefficient
    row1 = seadata.iloc[block1 - 1, 1:]
    row2 = seadata.iloc[block2 - 1, 1:]
    
    row1_mean = find_mean(row1)
    row1_sd = find_sd(row1, row1_mean)
    row2_mean = find_mean(row2)
    row2_sd = find_sd(row2, row2_mean)
    
    uppersum = 0
    
    for data1, data2 in zip(row1, row2):
        if (data1 == "-99999" or data1 == "XXXXX" or data2 == "-99999" or data2 == "XXXXX"):
            pass
        else:
            uppersum += (int(data1) - row1_mean) * (int(data2) - row2_mean)
            
    return uppersum / (row1_sd * row2_sd)
            

In [96]:
def linear_regression(block1, block2):
    
    row1 = seadata.iloc[block1 - 1, 1:]
    row2 = seadata.iloc[block2 - 1, 1:]
    
    #row1 will be on the Y-axis and row2 will be on the X-axis
    #first input block as Y axis, second candidate block  as X axis
    x_axis = []
    y_axis = []
    
    #extract valid data
    for data1, data2 in zip(row1, row2):
        if (data1 == "-99999" or data1 == "XXXXX" or data2 == "-99999" or data2 == "XXXXX"):
            pass
        else:
            #first input block as Y axis, second candidate block  as X axis
            y_axis.append(int(data1))
            x_axis.append(int(data2))
    
    #find regression
    line = np.polyfit(x = x_axis, y = y_axis, deg = 1)
    
    return list(line)

In [97]:
def find_similar_block():
    #result = (block, [(block, scire)similar list])
    result = []
    
    for block in range(1,21):
        score_list = [] #append(blockB, similarities)

        for waiting_block in range(1, 21):
            score_list.append((similarities(block, waiting_block), 
                                waiting_block, 
                               linear_regression(block, waiting_block)))
            
        score_list.sort(reverse = True)
            
        result.append((block, score_list))
    
    return result    

In [98]:
result = find_similar_block()

C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Users\Administrator\Anaconda3\l

In [99]:
def shorten_candidate(result):
    #this program shorten the similar block but seeting a threshold correlation value
    #between two block
    threshold_R = 0.065
    shorten_result = []
    
    #data is in the form
    #by all means, y = a*x + b
    #(block, [(R number, candidate_block, [a, b]).....(R number, candidate_block, [a, b])])
    for data in result:
        block = data[0]
        
        #maxium R value
        max_R = data[1][1][0]
        temp_candidate_detail_data = []
        
        #data[1] is in the form
        #[(R number, candidate_block, [a, b]).....(R number, candidate_block, [a, b])]
        #excluding the first candidate data as it is block itself
        for candidate_detail_data in data[1][1:]:
            
            #candidate_detail_data is in the form
            #(R number, candidate_block, [a, b])
            candidate_block = candidate_detail_data[1]
            R = candidate_detail_data[0]
            if max_R - R > threshold_R:
                break
            else:
                temp_candidate_detail_data.append(candidate_detail_data)
        
        shorten_result.append((block, temp_candidate_detail_data))
        
    return shorten_result

In [100]:
shorten_result = shorten_candidate(result)

In [109]:
shorten_result

[(1,
  [(0.90142673779444, 3, [0.75190494977259004, 52.67809970832986]),
   (0.8930462391045597, 6, [0.77641802188284181, 0.30062167702055964])]),
 (2,
  [(0.5481293046487008, 4, [0.88961974424487267, 33.773136154243048]),
   (0.5425178567338126, 13, [0.8940416142379668, -211.7274803294728]),
   (0.5346393035633833, 5, [0.89697772304817924, 31.721332350205774]),
   (0.48656033145669236, 16, [0.77348614972659779, -199.24541872901642])]),
 (3,
  [(0.9126545966026145, 6, [0.99762377317006434, -77.507628161718728]),
   (0.90142673779444, 1, [1.1220993049677688, -104.17730263576196]),
   (0.8560089162534658, 10, [0.99073154675104291, -82.73468291609133])]),
 (4, [(0.8519263494431825, 5, [0.94255887960145279, -42.516656982582184])]),
 (5,
  [(0.8623311012300217, 6, [1.0281920872582455, -175.80323211738303]),
   (0.8519263494431825, 4, [1.010477900539273, 23.4517028199805]),
   (0.8419826014884239, 3, [0.93461471422470199, -123.42799238316947]),
   (0.8286410687005241, 1, [1.0839959111326731,

# fill out the data

In [101]:
def extract_line_set(detail_data):
    #this part create the line set for the missing data
    
    line_set = []
    #data[1] is in the form
    #[(R number, candidate_block, [a, b]).....(R number, candidate_block, [a, b])]
    #excluding the first candidate data as it is block itself
    for candidate_detail_data in detail_data:
        #detail_data is in the form
        #(R number, candidate_block, [a, b])
        candidate_block = candidate_detail_data[1]
        a = candidate_detail_data[2][0]
        b = candidate_detail_data[2][1]
        line_set.append((candidate_block, a, b))
    
      
    return line_set

In [102]:
def reparing_order():
    #this function count the number of missing data in each row
    #then return a list that least missing value is at first
    order_list = []
    for block_index in range(20):
        num_missing = 0
        for column in range(1,300):
            cell_data = seadata.iloc[block_index, column]
            if cell_data == "-99999" or cell_data == "XXXXX":
                num_missing +=1
        
        order_list.append((block_index + 1, num_missing))
        
    order_list.sort(key = itemgetter(1))
    result_list = [x for (x,y) in order_list]
    
    print (result_list)
    return result_list
    

In [103]:
def twenty_mean(column):
    #return the mean value of 20 block in a given column
    column_sum = 0
    column_valid = 0
    for block_index in range(20):
        data = seadata.iloc[block_index, column]
        if (data == "-99999" or data == "XXXXX"):
            pass
        else:
            column_sum += float(data)
            column_valid += 1
    
    #this is the limitation of our data modelling
    #parit 8 and 11were used twenty mean value method
    #however, 8 and 11 means there isnt much sensor being deployed
    #we could say that they're less risky to be watered
    #therefore it is acceptable
    return column_sum / column_valid

In [104]:
def replace_missing(line_set, column, block):
    #replace a single cell data
    column_sum = 0
    valid_column = 0
    
    #by find the mean of its y=ax+b set data
    #where x equal to the candidate_block's data on that column
    #line set is in the form
    #[(related_block, a, b)]
    for (candidate_block, a, b) in line_set:
        candidate_block_index = candidate_block - 1
        candidate_cell_data = seadata.iloc[candidate_block_index, column]
        
        
        if (candidate_cell_data == "-99999" or candidate_cell_data == "XXXXX"):
            pass
        else:
            column_sum += a*float(candidate_cell_data) + b
            valid_column +=1
        
    if valid_column == 0:
        #in this extremely bade case(for block 8 and 11 only)
        #we replace this cell data with the mean value of 20 block
        temp = [candidate_block for (candidate_block, a, b) in line_set]
        print(block, column, temp)
        return twenty_mean(column)
        
        
    return column_sum / valid_column

In [105]:
def data_set_cleaning(result):
    #this data repair the whole data set
    
    #data is in the form
    #by all means, y = a*x + b
    #(block, [(R number, candidate_block, [a, b]).....(R number, candidate_block, [a, b])])
    least_missing = reparing_order() 
    
    #swap the order of 16 and 13, 11 and 8
    #so thar we can still using lin_modeling-mean method
    #as 16 may only be repaired by using data of 13
    #but 13 can be repaired by other block
    #therefore, we can repair 13 first
    least_missing[-1] = 16
    least_missing[-2] = 13
    
    
    for block in least_missing:
        block_index = block - 1
        data = result[block_index]
        block = data[0] #which is the main block
        
        line_set = extract_line_set(data[1]) #a set of linear regresiion data against this block
        
        for column in range(1,300):
            block_index = block - 1
            cell_data = seadata.iloc[block_index, column]
            if cell_data == "-99999" or cell_data == "XXXXX":
                seadata.iloc[block_index, column] = replace_missing(line_set, column, block)
                #print(seadata.iloc[block_index, column])
        
        
        
            
    
            

In [106]:
data_set_cleaning(shorten_result)

[1, 6, 19, 3, 10, 15, 20, 17, 5, 14, 7, 18, 9, 4, 8, 12, 11, 2, 16, 13]
11 1 [8]
11 2 [8]
11 3 [8]
11 4 [8]
11 5 [8]
11 6 [8]
11 7 [8]
11 8 [8]
11 9 [8]
11 10 [8]
11 11 [8]
11 12 [8]
11 13 [8]
11 14 [8]
11 15 [8]
11 16 [8]
11 17 [8]
11 18 [8]
11 19 [8]
11 20 [8]
11 21 [8]
11 22 [8]
11 23 [8]
11 24 [8]
11 25 [8]
11 26 [8]
11 27 [8]
11 28 [8]
11 29 [8]
11 30 [8]
11 31 [8]
11 32 [8]
11 33 [8]
11 34 [8]
11 35 [8]
11 36 [8]
11 37 [8]
11 38 [8]
11 39 [8]
11 40 [8]
11 41 [8]
11 42 [8]
11 43 [8]
11 44 [8]
11 45 [8]
11 46 [8]
11 47 [8]
11 48 [8]
11 49 [8]
11 50 [8]
11 51 [8]
11 52 [8]
11 53 [8]
11 54 [8]
11 55 [8]
11 56 [8]
11 57 [8]
11 58 [8]
11 59 [8]
11 60 [8]
11 61 [8]
11 62 [8]
11 63 [8]
11 64 [8]
11 65 [8]
11 66 [8]
11 67 [8]
11 68 [8]
11 69 [8]
11 70 [8]
11 71 [8]
11 72 [8]
11 73 [8]
11 74 [8]
11 75 [8]
11 76 [8]
11 77 [8]
11 78 [8]
11 79 [8]
11 80 [8]
11 81 [8]
11 82 [8]
11 83 [8]
11 84 [8]
11 85 [8]
11 86 [8]
11 87 [8]
11 88 [8]
11 89 [8]
11 90 [8]
11 91 [8]
11 92 [8]
11 93 [8]
11 94 [

In [107]:
seadata

,Year,1992,1992.1,1992.2,1992.3,1992.4,1992.5,1992.6,1992.7,1992.8,...,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,2016.8,2016.9,2016.10,2016.11
1,1,-247,-351,-300,-142,-45,-226,-340,-316,-97,...,-212,-130,-90,-155,-237,-161,-23,111,136,0
2,2,-438,-479.306,-399,-195,-143,-255,-429,-441,-225,...,-368.02,-263.528,-172.044,-244.197,-337.143,-278.195,-160.173,-65.8724,-59.8466,0
3,3,-455,-556,-458,-290,-165,-293,-458,-464,-254,...,-317,-184,-24,-122,-264,-210,-82,30,7.89221,0
4,4,-447,-548,-487,-289,-197,-362,-523,-508,-267,...,-457.292,-343.792,-244.421,-322.794,-423.754,-359.724,-231.527,-129.096,-122.551,0
5,5,-483,-587,-495,-318,-209,-364,-507,-483,-251,...,-440.053,-319.636,-214.209,-297.358,-404.471,-336.538,-200.529,-91.8559,-84.9118,0
6,6,-250,-360,-284,-113,-15,-186,-339,-326,-107,...,-265,-121,-1,-86,-183,-114,21,95,109,0
7,7,-246.986,-351.525,-279.298,-116.787,-23.6516,-186.163,-331.568,-319.213,-111.085,...,-239,-86,14,-48,-182,-145,-22,37,100,0
8,8,-249.928,-320.711,-251.234,-146.808,-72.4608,-170.841,-229.263,-223.056,-109.749,...,-231,-98,14,-92,-205,-142,7,100,123,0
9,9,-185.398,-276.401,-213.526,-72.0566,9.01947,-132.45,-259.028,-248.273,-67.0927,...,-217,-91,12,-81,-211,-126,26,144,145,0
10,10,-330,-381,-302,-183,-74,-199,-281,-275,-135,...,-234,-124,-58,-97,-137,-70,-13,52,27,0


In [108]:
seadata.to_csv('Complete data set.csv')